# Verificação de inconsistências lógicas em banco de dados 

Os códigos a seguir são uma adaptação para Python dos códigos originalmente escritos em VBA para detecção de inconsistências lógicas em bancos de dados de pesquisas cadastrais. 


Bibliotecas utilizadas: 
- pandas 
- openpyxl
- win32com (ambiente deve ter acesso ao word para validação ortográfica)

### Importando as planilhas que iremos trabalhar

In [2]:
import pandas as pd

''' NS: Ponderar sobre utilizar essas variáveis posteriormente para ajustar o caminho do arquivo importado sem alterar o código
# Caminho dos arquivos Excel
caminho_banco_de_dados = input('Insira o caminho do arquivo excel que será verificado: ')
caminho_matriz = input('Insira o caminho do arquivo "matriz": ')
caminho_intervalos = input('Insira o caminho do arquivo que contém os intervalos: ')
caminho_nomes = input('insira o caminho do arquivo que contenha a base de nomes válidos: ')
'''

## NS: Inserir Try/Excepts abaixo para casos em que não são necessários importar todas as planilhas acimas

# Importar as planilhas para DataFrames do pandas
df_bd = pd.read_excel(r"C:\Users\Nunes\OneDrive - NMC PROJETOS E CONSULTORIA LTDA\Área de Trabalho\Validação de Dados\base teste.xlsx") 
df_matriz = pd.read_excel(r"C:\Users\Nunes\OneDrive - NMC PROJETOS E CONSULTORIA LTDA\Área de Trabalho\Validação de Dados\matriz teste.xlsx", header=1)
df_intervalos = pd.read_excel(r"C:\Users\Nunes\OneDrive - NMC PROJETOS E CONSULTORIA LTDA\Área de Trabalho\Validação de Dados\intervalos teste.xlsx")
df_nomes = pd.read_excel(r"C:\Users\Nunes\OneDrive - NMC PROJETOS E CONSULTORIA LTDA\Área de Trabalho\Validação de Dados\nomes teste.xlsx")

# Exibir os primeiros registros para verificar se a importação foi bem-sucedida 
## NS: Alterar futuramente por msgs simples de confirmação de importação

'''
print("Banco de Dados:")
print(df_banco_de_dados.head())

print("\nMatriz:")
print(df_matriz.head())

print("\nIntervalos:")
print(df_intervalos.head())

print("\nNomes:")
print(df_matriz.head())
'''


'\nprint("Banco de Dados:")\nprint(df_banco_de_dados.head())\n\nprint("\nMatriz:")\nprint(df_matriz.head())\n\nprint("\nIntervalos:")\nprint(df_intervalos.head())\n\nprint("\nNomes:")\nprint(df_matriz.head())\n'

### Análise de preenchimento obrigatório

In [3]:
def AnalisePreenchimento(df_bd, df_matriz):
    resultado = []

    for coluna in range(1, len(df_matriz.columns)):  # Itera sobre as colunas da matriz
        texto = df_matriz.columns[coluna]  # Obtém o nome da coluna na matriz
        regra = df_matriz.iloc[4, coluna]  # Obtém a regra correspondente na linha 4 da matriz

        if texto in df_bd.columns:
            for linha in range(len(df_bd)):
                valor_celula = df_bd.loc[linha, texto]

                if (regra == "OBRIGATÓRIO" and pd.isna(valor_celula)) or (regra == "VAZIO" and not pd.isna(valor_celula)):
                    motivo = "Preenchimento obrigatório" if regra == "OBRIGATÓRIO" else "Sem preenchimento"
                    resultado.append({
                        'Data_Hora': pd.Timestamp.now(),
                        'Linha': linha+1,
                        'Coluna': texto,
                        'Motivo': motivo,
                        'Regra': regra,
                    
                    })

    resultado = pd.DataFrame(resultado)
    return resultado


In [4]:
resultado = AnalisePreenchimento(df_bd,df_matriz)
preenchimento = pd.DataFrame(resultado)
preenchimento.head()

,Data_Hora,Linha,Coluna,Motivo,Regra
0,2024-07-02 09:28:50.031280,9,CADASTRADOR,Preenchimento obrigatório,OBRIGATÓRIO
1,2024-07-02 09:28:50.032278,12,NOME DO ENTREVISTADO,Preenchimento obrigatório,OBRIGATÓRIO
2,2024-07-02 09:28:50.032278,1,ENDEREÇO,Preenchimento obrigatório,OBRIGATÓRIO


### validação de tipo

In [5]:
def AnalisarTipos(df_bd, df_matriz):
    resultados = []
    
    # Loop através das colunas da matriz 
    for coluna in range(2, len(df_matriz.columns)):
        texto = df_matriz.columns[coluna] 
        parametro = df_matriz.iloc[5, coluna]  
        
        if texto in df_bd.columns:
            if parametro == "TEXTO ABC":
                # Verificar se a coluna correspondente em df_bd contém apenas caracteres alfabéticos, espaços e acentuações
                filtro_alfabetico = df_bd[texto].apply(lambda x: str(x).replace(" ", "").isalpha())
                inconsistencias = df_bd[~filtro_alfabetico]  # ~filtro_alfabetico para pegar os que não são alfabéticos
            elif parametro == "DATA":
                # Verificar se a coluna correspondente em df_bd contém datas válidas
                filtro_data_invalida = pd.to_datetime(df_bd[texto], errors='coerce').isna()
                inconsistencias = df_bd[filtro_data_invalida]
            elif parametro == "NUMERO":
                # Verificar se a coluna correspondente em df_bd contém apenas números
                filtro_nao_numerico = pd.to_numeric(df_bd[texto], errors='coerce').isna()
                inconsistencias = df_bd[filtro_nao_numerico]
            else:
                inconsistencias = pd.DataFrame()

            # Para cada inconsistência encontrada, registre no dataframe de resultados
            for index, row in inconsistencias.iterrows():
                resultados.append({
                    'Data_Hora': pd.Timestamp.now(),
                    'Linha':index+1,
                    'Valor_Celula': row[texto],
                    'Parametro': parametro,
                    'Coluna': texto
                })
    
    return resultados

In [6]:
resultado_tipos = AnalisarTipos(df_bd,df_matriz)
tipos = pd.DataFrame(resultado_tipos)
tipos.head()

,Data_Hora,Linha,Valor_Celula,Parametro,Coluna
0,2024-07-02 09:28:50.086653,8,LUIZ],TEXTO ABC,NOME DO ENTREVISTADO


### Dicionário Ortográfico

In [7]:
import pandas as pd
import win32com.client as win32
from datetime import datetime

def main_verificar_ortografia(df_matriz, df_bd):
    erros_ortograficos = []

    # Verificar na linha 6 da "MATRIZ" onde está o termo "DICIONÁRIO ORTOGRÁFICO"
    for coluna in df_matriz.iloc[6].dropna().index:
        if df_matriz.iloc[6, df_matriz.columns.get_loc(coluna)] == "DICIONÁRIO ORTOGRÁFICO":
            # Para cada coluna encontrada, salvar o nome da coluna no df_matriz
            erros_ortograficos += verificar_ortografia(coluna, df_bd)

    # Converter a lista de dicionários em um DataFrame
    df_erros_ortograficos = pd.DataFrame(erros_ortograficos)
    return df_erros_ortograficos

def verificar_ortografia(nome_coluna, df_bd):
    WordApp = iniciar_aplicativo_word()
    erros_ortograficos = []

    try:
        # Validar ortografia em todas as linhas da coluna nome_coluna do df_bd
        for indice, valor in df_bd[nome_coluna].items():
            if pd.notnull(valor):  # Verifica se o valor não é nulo
                if not verificar_ortografia_word(valor, WordApp):
                    erros_ortograficos.append({
                        'Linha': indice+1,
                        'Data_Hora': datetime.now(),
                        'Mensagem': f"Inconsistência na célula {indice} da coluna '{nome_coluna}' da aba 'BD'",
                        'Texto_Incorreto': valor,
                        'Nome_Coluna': nome_coluna
                    })
    finally:
        fechar_aplicativo_word(WordApp)

    return erros_ortograficos

def iniciar_aplicativo_word():
    # Função para iniciar o aplicativo Microsoft Word
    WordApp = win32.Dispatch("Word.Application")
    WordApp.Visible = False  # Deixar o Word invisível
    return WordApp

def fechar_aplicativo_word(WordApp):
    # Função para fechar o aplicativo Microsoft Word
    if WordApp is not None:
        WordApp.Quit()

def verificar_ortografia_word(palavra, WordApp):
    # Função para verificar ortografia usando o Microsoft Word
    try:
        return WordApp.CheckSpelling(palavra)
    except Exception as e:
        print(f"Erro ao verificar ortografia: {e}")
        return True  # Retorna True para ignorar erros e continuar o processo (ajuste conforme necessário)


In [8]:
ortografia = main_verificar_ortografia(df_matriz,df_bd)
ortografia.head()

,Linha,Data_Hora,Mensagem,Texto_Incorreto,Nome_Coluna
0,5,2024-07-02 09:28:52.292285,Inconsistência na célula 4 da coluna 'LIVRE' d...,mu pi falow,LIVRE
1,14,2024-07-02 09:28:52.300490,Inconsistência na célula 13 da coluna 'LIVRE' ...,patacet mac,LIVRE


### Verificar nomes

In [9]:
import pandas as pd
from datetime import datetime

def verificar_nomes(df_matriz, df_bd, df_nomes):
    erros_encontrados = []

    # Verificar na linha 10 da "MATRIZ" onde está o termo "BANCO DE NOMES"
    for coluna in df_matriz.iloc[7].dropna().index:
        if df_matriz.iloc[7, df_matriz.columns.get_loc(coluna)] == "BANCO DE NOMES":
            # Iterar pelas linhas do df_bd na coluna correspondente
            for indice, valor in df_bd[coluna].items():
                if pd.notnull(valor):
                    # Verificar cada parte do nome individualmente
                    partes_nome = valor.split()
                    for parte_nome in partes_nome:
                        if not nome_eh_valido(parte_nome, df_nomes):
                            erros_encontrados.append({
                                'Data_Hora': datetime.now(),
                                'Linha': indice + 1,
                                'Texto_Incorreto': valor,
                                'Nome_Coluna': coluna
                            })

    # Converter a lista de dicionários em um DataFrame
    df_erros = pd.DataFrame(erros_encontrados, columns=['Data_Hora', 'Linha', 'Texto_Incorreto', 'Nome_Coluna'])

    return df_erros

def nome_eh_valido(nome, df_nomes):
    # Verificar se o nome está na coluna 0 do DataFrame df_nomes
    return nome in df_nomes.iloc[:, 0].values


In [10]:
nomes = verificar_nomes(df_matriz, df_bd, df_nomes)
nomes.head()

,Data_Hora,Linha,Texto_Incorreto,Nome_Coluna
0,2024-07-02 09:28:53.876156,2,PEDRON,NOME DO ENTREVISTADO
1,2024-07-02 09:28:53.880666,8,LUIZ],NOME DO ENTREVISTADO
2,2024-07-02 09:28:53.884699,13,TONHÃO,NOME DO ENTREVISTADO


### Verificar Intervalos

In [11]:
import pandas as pd
from datetime import datetime

# Lista para armazenar as inconsistências
inconsistencia_intervalo = []

row = df_matriz.iloc[8]

for col in df_matriz.columns:
    if 'intervalo' in str(row[col]):
        intervalo = str(row[col])
        if col in df_bd.columns:
            # Iterar sobre os valores da coluna correspondente no df_bd
            for idx, valor_bd in enumerate(df_bd[col]):
                try:
                    float(valor_bd)         ## NS: Maneira que encontrei de evitar que o código não reconheça o conteúdo devido o tipo
                    if valor_bd not in df_intervalos[intervalo]:
                        # Registrar inconsistência
                        hora_analise = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        inconsistencia_intervalo.append({
                            'Hora da analise': hora_analise,
                            'Coluna': col,
                            'Preenchimento': valor_bd,
                            'Linha': df_bd.index[idx]+1})
                except:
                    if str(valor_bd) not in str(df_intervalos[intervalo]):
                        # Registrar inconsistência
                        hora_analise = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        inconsistencia_intervalo.append({
                            'Hora da analise': hora_analise,
                            'Coluna': col,
                            'Preenchimento': valor_bd,
                            'Linha': df_bd.index[idx]+1})

# Criar DataFrame com os resultados das inconsistências
df_inconsistencias = pd.DataFrame(inconsistencia_intervalo).dropna()

# Exibir o DataFrame com os resultados das inconsistências
df_inconsistencias.head(30)


,Hora da analise,Coluna,Preenchimento,Linha
0,2024-07-02 09:28:53,IDADE DO ENTREVISTADO,150,9
2,2024-07-02 09:28:53,ENDEREÇO,RUA D,5
3,2024-07-02 09:28:53,CIDADE,CANADÁ,2


### Verificar Condicionais

In [55]:
import pandas as pd

df_matriz1 = df_matriz.fillna(value="") # NS: Corrige nan para o código identificar vazio, pode ter outra saída

coluna_condicionante = None
coluna_condicionada = None
dado_condicionante = None
resultado = None
linha = 15
incons = []

for linha in range(15,92,4):
    # Iterar sobre a linha 15 do df_matriz, a partir da coluna 2
    linha_matriz = df_matriz1.iloc[linha]
    for col in range(3, len(linha_matriz)):
        valor = linha_matriz.iloc[col]
        if valor != "":
            coluna_condicionante = valor
            coluna_condicionada = df_matriz1.columns[col]
            dado_condicionante = df_matriz1.iloc[linha+1, col]  # Linha abaixo onde foi encontrado o valor
            resultado = df_matriz1.iloc[linha+2, col]  # Duas linhas abaixo
            # Verificar no df_bd por inconsistências
            for indice, valor_bd in enumerate(df_bd[coluna_condicionante]):
                if valor_bd == dado_condicionante:
                    if df_bd[coluna_condicionada][indice] != resultado:
                        incons.append({
                            "Data e hora": datetime.now(),
                            "Linha": indice+1,
                            "Coluna condicionada": coluna_condicionada,
                            "Coluna condicionante":coluna_condicionante,
                            "Dado condicionante": dado_condicionante,
                            "Resultado": resultado,
                            "Valor": valor_bd })
                        


inconsistencias = pd.DataFrame(incons, columns=['Data e hora', "Linha", "Coluna condicionada", "Coluna condicionante", "Dado condicionante", "Resultado", "Valor"]) 
inconsistencias.head()

,Data e hora,Linha,Coluna condicionada,Coluna condicionante,Dado condicionante,Resultado,Valor
0,2024-07-02 11:00:59.780539,18,SEXO,GRAVIDEZ?,SIM,F,SIM
